In [102]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 3: Crossed Wires ---</h2><p>The gravity assist was successful, and you're well on your way to the Venus refuelling station.  During the rush back on Earth, the fuel management system wasn't completely installed, so that's next on the priority list.</p>
<p>Opening the front panel reveals a jumble of wires. Specifically, <em>two wires</em> are connected to a central port and extend outward on a grid.  You trace the path each wire takes as it leaves the central port, one wire per line of text (your puzzle input).</p>
<p>The wires <span title="A jumble of twisty little wires, all alike.">twist and turn</span>, but the two wires occasionally cross paths. To fix the circuit, you need to <em>find the intersection point closest to the central port</em>. Because the wires are on a grid, use the <a href="https://en.wikipedia.org/wiki/Taxicab_geometry">Manhattan distance</a> for this measurement. While the wires do technically cross right at the central port where they both start, this point does not count, nor does a wire count as crossing with itself.</p>
<p>For example, if the first wire's path is <code>R8,U5,L5,D3</code>, then starting from the central port (<code>o</code>), it goes right <code>8</code>, up <code>5</code>, left <code>5</code>, and finally down <code>3</code>:</p>
<pre><code>...........
...........
...........
....+----+.
....|....|.
....|....|.
....|....|.
.........|.
.o-------+.
...........
</code></pre>
<p>Then, if the second wire's path is <code>U7,R6,D4,L4</code>, it goes up <code>7</code>, right <code>6</code>, down <code>4</code>, and left <code>4</code>:</p>
<pre><code>...........
.+-----+...
.|.....|...
.|..+--X-+.
.|..|..|.|.
.|.-<em>X</em>--+.|.
.|..|....|.
.|.......|.
.o-------+.
...........
</code></pre>
<p>These wires cross at two locations (marked <code>X</code>), but the lower-left one is closer to the central port: its distance is <code>3 + 3 = 6</code>.</p>
<p>Here are a few more examples:</p>
<ul>
<li><code>R75,D30,R83,U83,L12,D49,R71,U7,L72<br>U62,R66,U55,R34,D71,R55,D58,R83</code> = distance <code>159</code></li>
<li><code>R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51<br>U98,R91,D20,R16,D67,R40,U7,R15,U6,R7</code> = distance <code>135</code></li>
</ul>
<p><em>What is the Manhattan distance</em> from the central port to the closest intersection?</p>
</article>


In [103]:
from more_itertools import first


tests = [
    {
        "name": "Example 1",
        "wires": "\n".join(("R8,U5,L5,D3", "U7,R6,D4,L4")),
        "expected": 6,
    },
    {
        "name": "Example 2",
        "wires": "\n".join(
            (
                "R75,D30,R83,U83,L12,D49,R71,U7,L72",
                "U62,R66,U55,R34,D71,R55,D58,R83",
            )
        ),
        "expected": 159,
    },
    {
        "name": "Example 3",
        "wires": "\n".join(
            (
                "R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51",
                "U98,R91,D20,R16,D67,R40,U7,R15,U6,R7",
            )
        ),
        "expected": 135,
    },
]


DIRECTIONS = {"R": (0, 1), "U": (-1, 0), "L": (0, -1), "D": (1, 0)}


def intersection_closest_to_port(wires: str) -> int:
    iter = (
        ((c[0], int(c[1:])) for c in l.split(",")) for l in wires.strip().splitlines()
    )
    counter = defaultdict(int)
    lines = defaultdict(set)
    for wire_id, wire in enumerate(iter):
        x, y = 0, 0
        for direction, steps in wire:
            dx, dy = DIRECTIONS[direction]
            for _ in range(steps):
                x += dx
                y += dy

                if (x, y) in lines[wire_id]:
                    continue

                counter[(x, y)] += 1
                lines[wire_id].add((x, y))

    return first(
        sorted(abs(x) + abs(y) for (x, y), count in counter.items() if count > 1)
    )


run_tests_params(intersection_closest_to_port, tests)


Test Example 1 passed, for intersection_closest_to_port.
Test Example 2 passed, for intersection_closest_to_port.
Test Example 3 passed, for intersection_closest_to_port.
Success


In [104]:
with open("../input/day3.txt") as f:
    puzzle = f.read()

print(f"Part I: {intersection_closest_to_port(puzzle)}")

Part I: 5319


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>5319</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>It turns out that this circuit is very timing-sensitive; you actually need to <em>minimize the signal delay</em>.</p>
<p>To do this, calculate the <em>number of steps</em> each wire takes to reach each intersection; choose the intersection where the <em>sum of both wires' steps</em> is lowest. If a wire visits a position on the grid multiple times, use the steps value from the <em>first</em> time it visits that position when calculating the total value of a specific intersection.</p>
<p>The number of steps a wire takes is the total number of grid squares the wire has entered to get to that location, including the intersection being considered. Again consider the example from above:</p>
<pre><code>...........
.+-----+...
.|.....|...
.|..+--X-+.
.|..|..|.|.
.|.-X--+.|.
.|..|....|.
.|.......|.
.o-------+.
...........
</code></pre>
<p>In the above example, the intersection closest to the central port is reached after <code>8+5+5+2 = <em>20</em></code> steps by the first wire and <code>7+6+4+3 = <em>20</em></code> steps by the second wire for a total of <code>20+20 = <em>40</em></code> steps.</p>
<p>However, the top-right intersection is better: the first wire takes only <code>8+5+2 = <em>15</em></code> and the second wire takes only <code>7+6+2 = <em>15</em></code>, a total of <code>15+15 = <em>30</em></code> steps.</p>
<p>Here are the best steps for the extra examples from above:</p>
<ul>
<li><code>R75,D30,R83,U83,L12,D49,R71,U7,L72<br>U62,R66,U55,R34,D71,R55,D58,R83</code> = <code>610</code> steps</li>
<li><code>R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51<br>U98,R91,D20,R16,D67,R40,U7,R15,U6,R7</code> = <code>410</code> steps</li>
</ul>
<p><em>What is the fewest combined steps the wires must take to reach an intersection?</em></p>
</article>

</main>


In [105]:
from more_itertools import first


tests = [
    {
        "name": "Example 1",
        "wires": "\n".join(("R8,U5,L5,D3", "U7,R6,D4,L4")),
        "expected": 30,
    },
    {
        "name": "Example 2",
        "wires": "\n".join(
            (
                "R75,D30,R83,U83,L12,D49,R71,U7,L72",
                "U62,R66,U55,R34,D71,R55,D58,R83",
            )
        ),
        "expected": 610,
    },
    {
        "name": "Example 3",
        "wires": "\n".join(
            (
                "R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51",
                "U98,R91,D20,R16,D67,R40,U7,R15,U6,R7",
            )
        ),
        "expected": 410,
    },
]


DIRECTIONS = {"R": (0, 1), "U": (-1, 0), "L": (0, -1), "D": (1, 0)}


def intersection_with_least_steps(wires: str) -> int:
    iter = (
        ((c[0], int(c[1:])) for c in l.split(",")) for l in wires.strip().splitlines()
    )
    counter = defaultdict(int)
    lines = defaultdict(dict)
    for wire_id, wire in enumerate(iter):
        x, y = 0, 0
        step = 0
        for direction, steps in wire:
            dx, dy = DIRECTIONS[direction]
            for _ in range(steps):
                x += dx
                y += dy
                step += 1

                if (x, y) in lines[wire_id]:
                    continue

                counter[(x, y)] += 1
                lines[wire_id][(x, y)] = step

    return first(
        sorted(
            sum(steps[(x, y)] for steps in lines.values())
            for (x, y), count in counter.items()
            if count > 1
        )
    )


run_tests_params(intersection_with_least_steps, tests)


Test Example 1 passed, for intersection_with_least_steps.
Test Example 2 passed, for intersection_with_least_steps.
Test Example 3 passed, for intersection_with_least_steps.
Success


In [106]:
print(f"Part II: {intersection_with_least_steps(puzzle)}")

Part II: 122514


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>122514</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
